# LlamaIndexの前準備

In [ ]:
# パッケージのインストール
!pip install llama-index==0.6.12

In [ ]:
# 環境変数の準備
import os
os.environ["OPENAI_API_KEY"] = "<OpenAI_APIのAPIキー>"

In [ ]:
import logging
import sys

# ログレベルの設定
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

# Faissを使った質問応答

In [ ]:
# パッケージのインストール
!pip install faiss-gpu

In [ ]:
from llama_index import SimpleDirectoryReader

# ドキュメントの読み込み (dataフォルダにドキュメントを配置しておきます)
documents = SimpleDirectoryReader("data").load_data()

In [ ]:
import faiss

# faissのインデックスの作成
faiss_index = faiss.IndexFlatL2(1536)

In [ ]:
from llama_index import GPTVectorStoreIndex, StorageContext
from llama_index.vector_stores.faiss import FaissVectorStore

# インデックスの作成
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context
)

In [ ]:
# クエリエンジンの作成
query_engine = index.as_query_engine()

In [ ]:
# 質問応答
print(query_engine.query("ミコの幼馴染の名前は？"))

# Pineconeを使った質問応答

In [ ]:
# パッケージのインストール
!pip install pinecone-client
!pip install transformers

In [ ]:
from llama_index import SimpleDirectoryReader

# ドキュメントの読み込み (dataフォルダにドキュメントを配置しておきます)
documents = SimpleDirectoryReader("data").load_data()

In [ ]:
import pinecone

# pinecone-clientのインデックスの生成
api_key = "<PineconeのAPIキー>"
pinecone.init(api_key=api_key, environment="us-west1-gcp")
pinecone.create_index(
    "quickstart",
    dimension=1536,
    metric="dotproduct",
    pod_type="p1"
)
pinecone_index = pinecone.Index("quickstart")

In [ ]:
from llama_index import GPTVectorStoreIndex, StorageContext
from llama_index.vector_stores.pinecone import PineconeVectorStore

# インデックスの作成
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context
)

In [ ]:
# クエリエンジンの作成
query_engine = index.as_query_engine()

In [ ]:
# 質問応答
print(query_engine.query("ミコの幼馴染の名前は？"))